## Fama-MacBetch

Implementing a fama-MacBeth 2-stage egression to estimate factor risk premia, make inference on the risk premia, and test whether a linear factor model can explain a crosssection of portfolio returns. 

Import required modules

In [1]:
from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import inv
from scipy.stats import chi2
import pandas as pd
import statsmodels.api as sm

In [2]:
data = pd.read_csv('FamaFrench.csv')

# Split using both named colums and iloc for larger blocks
dates = data['date'].values
factors = data[['VWMe', 'SMB', 'HML']].values
riskfree = data['RF'].values
portfolios = data.iloc[:, 5:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape

# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [3]:
data.head()

,date,VWMe,SMB,HML,RF,S1V1,S1V2,S1V3,S1V4,S1V5,...,S4V1,S4V2,S4V3,S4V4,S4V5,S5V1,S5V2,S5V3,S5V4,S5V5
0,192607,2.62,-2.16,-2.92,0.22,3.61,-3.69,-0.64,-1.42,-0.64,...,3.29,1.24,1.29,0.55,2.56,3.18,6.08,2.00,2.93,0.56
1,192608,2.56,-1.49,4.88,0.25,-1.94,-6.78,3.81,1.21,4.82,...,0.76,4.11,1.93,2.13,4.47,1.17,4.10,1.82,5.64,7.76
2,192609,0.36,-1.38,-0.01,0.23,-6.41,3.45,-5.19,3.08,0.75,...,1.87,-0.08,-1.84,1.56,2.18,-1.37,3.66,-0.23,-0.30,-2.43
3,192610,-3.43,0.04,0.71,0.32,-8.66,-10.02,-3.80,0.04,-3.06,...,0.13,-1.71,-2.33,-2.93,-5.21,-3.14,-3.13,-2.21,-4.59,-5.81
4,192611,2.44,-0.24,-0.31,0.31,3.77,12.42,2.31,-3.36,1.40,...,3.54,2.28,3.85,5.00,1.78,4.31,2.61,1.47,3.55,2.56


In [4]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)

# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [5]:
cs_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     55.79
Date:                Mon, 24 Jun 2019   Prob (F-statistic):           1.90e-10
Time:                        21:41:08   Log-Likelihood:                -5.4240
No. Observations:                  25   AIC:                             16.85
Df Residuals:                      22   BIC:                             20.50
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.5554      0.106      5.246      0.000       0.336       0.775
x2             0.2394      0.124      1.926      0.067      -0.018       0.497
x3             0.2340      0.169      1.384      0.180      -0.117       0.585
==============================================================================
Omnibus:                       29.159   Durbin-Watson:                   0.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.876
Skew:                          -2.392   Prob(JB):                     2.00e-12
Kurtosis:                       8.371   Cond. No.                         3.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp
vcv = inv(G.T) * S * inv(G) / T
vcvAlpha = vcv[0:N * K + N:4, 0:N * K + N:4]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(25).cdf(J)

In [7]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print(' Annualized Risk Premia')
print('          Market  SMB    HML')
print('--------------------------------------')
print('Premia   {0:0.4f} {1:0.4f} {2:0.4f}'.format(arp[0], arp[1], arp[2]))
print('Std.Err. {0:0.4f} {1:0.4f} {2:0.4f}'.format(arpSE[0], arpSE[1], arpSE[2]))
print('\n\n')
print('J-test: {:0.4f}'.format(J))
print('P-value: {:0.4f}'.format(Jpval))
i = 0
betaSE = []
for j in range(5):
    for k in range(5):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:} Alpha Beta(VWM) Beta(SMB) Beta(HML)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(a, b[0], b[1], b[2]))
        print('Std Err. {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(s[0], s[1], s[2], s[3]))
        print('T-stat {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(t[0], t[1], t[2], t[3]))
        print('')
        i += 1
betaSE = array(betaSE)
savez_compressed('fama-macbeth-results', alpha=alpha, beta=beta,
betaSE=betaSE, arpSE=arpSE, arp=arp, J=J, Jpval=Jpval)        

 Annualized Risk Premia
          Market  SMB    HML
--------------------------------------
Premia   6.6642 2.8731 2.8080
Std.Err. 0.5994 0.4010 0.4296



J-test: 95.2879
P-value: 0.0000
Size: 1, Value:1 Alpha Beta(VWM) Beta(SMB) Beta(HML)
Coefficients:    -0.8354     1.3099     1.2892     0.3943
Std Err.     0.1820     0.1269     0.1671     0.2748
T-stat    -4.5904    10.3196     7.7127     1.4348

Size: 1, Value:2 Alpha Beta(VWM) Beta(SMB) Beta(HML)
Coefficients:    -0.3911     1.0853     1.6100     0.3317
Std Err.     0.1237     0.0637     0.1893     0.1444
T-stat    -3.1616    17.0351     8.5061     2.2971

Size: 1, Value:3 Alpha Beta(VWM) Beta(SMB) Beta(HML)
Coefficients:    -0.1219     1.0747     1.1812     0.4648
Std Err.     0.0997     0.0419     0.0938     0.0723
T-stat    -1.2225    25.6206    12.5952     6.4310

Size: 1, Value:4 Alpha Beta(VWM) Beta(SMB) Beta(HML)
Coefficients:     0.0388     0.9630     1.2249     0.5854
Std Err.     0.0692     0.0232     0.1003     0.0353
T